<h2>Regression Models with Keras</h2>

### Read and Clean Dataset

In [40]:
import pandas as pd
import numpy as np

In [41]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [31]:
concrete_data.shape

(1030, 9)

In [32]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


concrete_data.isnull().sum()

### Split data into predictors and target

In [33]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [34]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [35]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [36]:
predictors.head(10)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
5,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90
6,380.0,95.0,0.0,228.0,0.0,932.0,594.0,365
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28
8,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28
9,475.0,0.0,0.0,228.0,0.0,932.0,594.0,28


In [37]:
cols = predictors.shape[1] # number of predictors
cols

8

## Import Keras


In [42]:
! pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.4 MB 435.7 kB/s eta 0:00:20
     ---------------------------------------- 0.1/8.4 MB 656.4 kB/s eta 0:00:13
      --------------------------------------- 0.1/8.4 MB 804.6 kB/s eta 0:00:11
      --------------------------------------- 0.1/8.4 MB 853.3 kB/s eta 0:00:10
      --------------------------------------- 0.2/8.4 MB 751.6 kB/s eta 0:00:11
      --------------------------------------- 0.2/8.4 MB 737.3 kB/s eta 0:00:12
      --------------------------------------- 0.2/8.4 MB 593.2 kB/s eta 0:00:14
     - -------------------------------------- 0.3/8.4 MB 791.9 kB/s eta 0:00:11
     - -------------------------------------- 0.3/8.4 MB 807.1 kB/s eta 0:00:10
     - -------------------------------------- 0.4/8.4 MB 859.0 kB/s eta 0:00:10
     -- ------------------------------------- 0.4/8

In [43]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import sklearn

## Build a Neural Network


In [44]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [45]:
# build the model
model = regression_model()

### split the data into a training and test sets

In [69]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)

In [70]:
len(X_train)

690

In [71]:
len(X_test)

340

In [72]:
# fit the model
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
16/16 - 0s - loss: 339.9896 - val_loss: 335.1876 - 130ms/epoch - 8ms/step
Epoch 2/50
16/16 - 0s - loss: 335.1440 - val_loss: 340.0556 - 80ms/epoch - 5ms/step
Epoch 3/50
16/16 - 0s - loss: 324.2996 - val_loss: 324.1505 - 71ms/epoch - 4ms/step
Epoch 4/50
16/16 - 0s - loss: 318.8000 - val_loss: 318.1987 - 80ms/epoch - 5ms/step
Epoch 5/50
16/16 - 0s - loss: 311.9309 - val_loss: 314.9512 - 63ms/epoch - 4ms/step
Epoch 6/50
16/16 - 0s - loss: 306.1090 - val_loss: 307.4756 - 67ms/epoch - 4ms/step
Epoch 7/50
16/16 - 0s - loss: 303.2624 - val_loss: 301.7616 - 67ms/epoch - 4ms/step
Epoch 8/50
16/16 - 0s - loss: 295.8118 - val_loss: 298.8665 - 109ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 291.4248 - val_loss: 291.0295 - 88ms/epoch - 6ms/step
Epoch 10/50
16/16 - 0s - loss: 284.6929 - val_loss: 285.8238 - 81ms/epoch - 5ms/step
Epoch 11/50
16/16 - 0s - loss: 280.2499 - val_loss: 284.2653 - 90ms/epoch - 6ms/step
Epoch 12/50
16/16 - 0s - loss: 276.5319 - val_loss: 275.5172 - 72ms/epoc

In [73]:
y_pred =model.predict(X_test)

11/11 [==============================] - 0s 2ms/step


### mean_squared_error

In [74]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

149.65711803963114

### 50 mean squared errors

In [81]:
x=0
MSE_list =[]
while(x<50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=2)
    y_pred =model.predict(X_test)
    MSE_list.append(mean_squared_error(y_test, y_pred))
    x=x+1

Epoch 1/50
16/16 - 0s - loss: 149.3537 - val_loss: 147.2109 - 145ms/epoch - 9ms/step
Epoch 2/50
16/16 - 0s - loss: 148.6139 - val_loss: 144.0780 - 89ms/epoch - 6ms/step
Epoch 3/50
16/16 - 0s - loss: 146.8582 - val_loss: 142.5542 - 82ms/epoch - 5ms/step
Epoch 4/50
16/16 - 0s - loss: 145.4108 - val_loss: 141.2570 - 77ms/epoch - 5ms/step
Epoch 5/50
16/16 - 0s - loss: 147.8482 - val_loss: 140.4583 - 70ms/epoch - 4ms/step
Epoch 6/50
16/16 - 0s - loss: 143.7573 - val_loss: 139.3312 - 68ms/epoch - 4ms/step
Epoch 7/50
16/16 - 0s - loss: 141.9956 - val_loss: 138.2816 - 81ms/epoch - 5ms/step
Epoch 8/50
16/16 - 0s - loss: 140.8246 - val_loss: 135.9124 - 67ms/epoch - 4ms/step
Epoch 9/50
16/16 - 0s - loss: 143.7320 - val_loss: 135.2172 - 67ms/epoch - 4ms/step
Epoch 10/50
16/16 - 0s - loss: 139.7521 - val_loss: 132.6279 - 82ms/epoch - 5ms/step
Epoch 11/50
16/16 - 0s - loss: 139.5538 - val_loss: 131.9843 - 79ms/epoch - 5ms/step
Epoch 12/50
16/16 - 0s - loss: 138.2304 - val_loss: 130.5198 - 83ms/epoch

Epoch 47/50
16/16 - 0s - loss: 115.8492 - val_loss: 108.1109 - 65ms/epoch - 4ms/step
Epoch 48/50
16/16 - 0s - loss: 111.8563 - val_loss: 107.6453 - 93ms/epoch - 6ms/step
Epoch 49/50
16/16 - 0s - loss: 110.9278 - val_loss: 108.6632 - 77ms/epoch - 5ms/step
Epoch 50/50
16/16 - 0s - loss: 111.4240 - val_loss: 109.5272 - 71ms/epoch - 4ms/step
11/11 [==============================] - 0s 2ms/step
Epoch 1/50
16/16 - 0s - loss: 111.8051 - val_loss: 107.3901 - 135ms/epoch - 8ms/step
Epoch 2/50
16/16 - 0s - loss: 110.9725 - val_loss: 107.9675 - 95ms/epoch - 6ms/step
Epoch 3/50
16/16 - 0s - loss: 109.9788 - val_loss: 110.3677 - 68ms/epoch - 4ms/step
Epoch 4/50
16/16 - 0s - loss: 113.4802 - val_loss: 110.7319 - 82ms/epoch - 5ms/step
Epoch 5/50
16/16 - 0s - loss: 110.9081 - val_loss: 108.6554 - 68ms/epoch - 4ms/step
Epoch 6/50
16/16 - 0s - loss: 109.6895 - val_loss: 109.6809 - 81ms/epoch - 5ms/step
Epoch 7/50
16/16 - 0s - loss: 112.8301 - val_loss: 109.7430 - 89ms/epoch - 6ms/step
Epoch 8/50
16/16 -

Epoch 43/50
16/16 - 0s - loss: 110.2189 - val_loss: 110.9905 - 79ms/epoch - 5ms/step
Epoch 44/50
16/16 - 0s - loss: 114.6142 - val_loss: 108.9147 - 68ms/epoch - 4ms/step
Epoch 45/50
16/16 - 0s - loss: 113.0546 - val_loss: 122.9161 - 83ms/epoch - 5ms/step
Epoch 46/50
16/16 - 0s - loss: 113.2239 - val_loss: 114.7615 - 68ms/epoch - 4ms/step
Epoch 47/50
16/16 - 0s - loss: 111.2894 - val_loss: 112.2629 - 82ms/epoch - 5ms/step
Epoch 48/50
16/16 - 0s - loss: 111.8179 - val_loss: 108.8236 - 68ms/epoch - 4ms/step
Epoch 49/50
16/16 - 0s - loss: 109.1605 - val_loss: 112.7029 - 67ms/epoch - 4ms/step
Epoch 50/50
16/16 - 0s - loss: 113.9894 - val_loss: 113.5598 - 67ms/epoch - 4ms/step
11/11 [==============================] - 0s 2ms/step
Epoch 1/50
16/16 - 0s - loss: 108.0031 - val_loss: 108.2673 - 130ms/epoch - 8ms/step
Epoch 2/50
16/16 - 0s - loss: 112.2308 - val_loss: 110.4251 - 90ms/epoch - 6ms/step
Epoch 3/50
16/16 - 0s - loss: 110.4357 - val_loss: 108.4317 - 99ms/epoch - 6ms/step
Epoch 4/50
16/

Epoch 39/50
16/16 - 0s - loss: 108.6087 - val_loss: 107.2794 - 74ms/epoch - 5ms/step
Epoch 40/50
16/16 - 0s - loss: 111.9821 - val_loss: 108.6456 - 82ms/epoch - 5ms/step
Epoch 41/50
16/16 - 0s - loss: 110.1780 - val_loss: 108.2465 - 67ms/epoch - 4ms/step
Epoch 42/50
16/16 - 0s - loss: 115.9362 - val_loss: 125.3874 - 81ms/epoch - 5ms/step
Epoch 43/50
16/16 - 0s - loss: 113.2967 - val_loss: 114.9966 - 67ms/epoch - 4ms/step
Epoch 44/50
16/16 - 0s - loss: 116.0925 - val_loss: 128.8571 - 67ms/epoch - 4ms/step
Epoch 45/50
16/16 - 0s - loss: 118.6818 - val_loss: 115.9378 - 68ms/epoch - 4ms/step
Epoch 46/50
16/16 - 0s - loss: 109.9896 - val_loss: 106.8949 - 73ms/epoch - 5ms/step
Epoch 47/50
16/16 - 0s - loss: 108.3105 - val_loss: 107.5799 - 58ms/epoch - 4ms/step
Epoch 48/50
16/16 - 0s - loss: 110.9091 - val_loss: 111.5799 - 103ms/epoch - 6ms/step
Epoch 49/50
16/16 - 0s - loss: 128.5060 - val_loss: 138.5802 - 111ms/epoch - 7ms/step
Epoch 50/50
16/16 - 0s - loss: 134.5431 - val_loss: 129.5989 - 

Epoch 35/50
16/16 - 0s - loss: 121.2534 - val_loss: 115.2265 - 62ms/epoch - 4ms/step
Epoch 36/50
16/16 - 0s - loss: 113.0760 - val_loss: 110.1976 - 83ms/epoch - 5ms/step
Epoch 37/50
16/16 - 0s - loss: 119.2590 - val_loss: 126.5795 - 66ms/epoch - 4ms/step
Epoch 38/50
16/16 - 0s - loss: 113.0605 - val_loss: 104.4493 - 68ms/epoch - 4ms/step
Epoch 39/50
16/16 - 0s - loss: 111.2087 - val_loss: 106.6708 - 65ms/epoch - 4ms/step
Epoch 40/50
16/16 - 0s - loss: 117.2381 - val_loss: 107.0719 - 67ms/epoch - 4ms/step
Epoch 41/50
16/16 - 0s - loss: 112.0347 - val_loss: 115.3358 - 77ms/epoch - 5ms/step
Epoch 42/50
16/16 - 0s - loss: 116.3421 - val_loss: 104.5122 - 69ms/epoch - 4ms/step
Epoch 43/50
16/16 - 0s - loss: 108.3153 - val_loss: 111.1088 - 98ms/epoch - 6ms/step
Epoch 44/50
16/16 - 0s - loss: 111.0328 - val_loss: 105.6243 - 69ms/epoch - 4ms/step
Epoch 45/50
16/16 - 0s - loss: 121.4783 - val_loss: 113.3922 - 81ms/epoch - 5ms/step
Epoch 46/50
16/16 - 0s - loss: 122.4686 - val_loss: 110.6310 - 82

...

Epoch 25/50
16/16 - 0s - loss: 62.0975 - val_loss: 65.0745 - 86ms/epoch - 5ms/step
Epoch 26/50
16/16 - 0s - loss: 61.7378 - val_loss: 64.4454 - 76ms/epoch - 5ms/step
Epoch 27/50
16/16 - 0s - loss: 66.8426 - val_loss: 81.5347 - 81ms/epoch - 5ms/step
Epoch 28/50
16/16 - 0s - loss: 68.5151 - val_loss: 68.8855 - 62ms/epoch - 4ms/step
Epoch 29/50
16/16 - 0s - loss: 61.7463 - val_loss: 63.9210 - 78ms/epoch - 5ms/step
Epoch 30/50
16/16 - 0s - loss: 61.4403 - val_loss: 66.0979 - 62ms/epoch - 4ms/step
Epoch 31/50
16/16 - 0s - loss: 67.2133 - val_loss: 67.3008 - 62ms/epoch - 4ms/step
Epoch 32/50
16/16 - 0s - loss: 59.9916 - val_loss: 68.1913 - 78ms/epoch - 5ms/step
Epoch 33/50
16/16 - 0s - loss: 62.9193 - val_loss: 64.0348 - 78ms/epoch - 5ms/step
Epoch 34/50
16/16 - 0s - loss: 65.0828 - val_loss: 70.7599 - 115ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 71.2172 - val_loss: 79.9737 - 74ms/epoch - 5ms/step
Epoch 36/50
16/16 - 0s - loss: 68.8759 - val_loss: 66.0422 - 67ms/epoch - 4ms/step
Epo

Epoch 23/50
16/16 - 0s - loss: 61.4908 - val_loss: 61.5862 - 101ms/epoch - 6ms/step
Epoch 24/50
16/16 - 0s - loss: 59.6225 - val_loss: 64.5265 - 83ms/epoch - 5ms/step
Epoch 25/50
16/16 - 0s - loss: 59.4792 - val_loss: 73.2027 - 71ms/epoch - 4ms/step
Epoch 26/50
16/16 - 0s - loss: 63.1721 - val_loss: 64.2754 - 67ms/epoch - 4ms/step
Epoch 27/50
16/16 - 0s - loss: 59.5746 - val_loss: 63.0320 - 62ms/epoch - 4ms/step
Epoch 28/50
16/16 - 0s - loss: 59.1646 - val_loss: 63.8688 - 73ms/epoch - 5ms/step
Epoch 29/50
16/16 - 0s - loss: 60.0073 - val_loss: 64.7890 - 77ms/epoch - 5ms/step
Epoch 30/50
16/16 - 0s - loss: 67.3350 - val_loss: 80.4345 - 67ms/epoch - 4ms/step
Epoch 31/50
16/16 - 0s - loss: 74.8319 - val_loss: 70.1324 - 88ms/epoch - 6ms/step
Epoch 32/50
16/16 - 0s - loss: 63.9579 - val_loss: 65.5012 - 73ms/epoch - 5ms/step
Epoch 33/50
16/16 - 0s - loss: 61.1132 - val_loss: 63.3494 - 111ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 62.5727 - val_loss: 63.1352 - 87ms/epoch - 5ms/step
Ep

Epoch 21/50
16/16 - 0s - loss: 59.9944 - val_loss: 67.1101 - 80ms/epoch - 5ms/step
Epoch 22/50
16/16 - 0s - loss: 62.0157 - val_loss: 66.3876 - 68ms/epoch - 4ms/step
Epoch 23/50
16/16 - 0s - loss: 58.8913 - val_loss: 63.5334 - 66ms/epoch - 4ms/step
Epoch 24/50
16/16 - 0s - loss: 59.6964 - val_loss: 64.1555 - 66ms/epoch - 4ms/step
Epoch 25/50
16/16 - 0s - loss: 58.9172 - val_loss: 62.4142 - 83ms/epoch - 5ms/step
Epoch 26/50
16/16 - 0s - loss: 58.9673 - val_loss: 77.3724 - 84ms/epoch - 5ms/step
Epoch 27/50
16/16 - 0s - loss: 67.7275 - val_loss: 67.1797 - 90ms/epoch - 6ms/step
Epoch 28/50
16/16 - 0s - loss: 59.8833 - val_loss: 78.4092 - 73ms/epoch - 5ms/step
Epoch 29/50
16/16 - 0s - loss: 63.6258 - val_loss: 70.3117 - 84ms/epoch - 5ms/step
Epoch 30/50
16/16 - 0s - loss: 60.1400 - val_loss: 66.7713 - 83ms/epoch - 5ms/step
Epoch 31/50
16/16 - 0s - loss: 62.4884 - val_loss: 63.0560 - 78ms/epoch - 5ms/step
Epoch 32/50
16/16 - 0s - loss: 58.5107 - val_loss: 67.5884 - 69ms/epoch - 4ms/step
Epoc

Epoch 19/50
16/16 - 0s - loss: 62.3205 - val_loss: 64.1890 - 57ms/epoch - 4ms/step
Epoch 20/50
16/16 - 0s - loss: 61.2901 - val_loss: 62.1788 - 73ms/epoch - 5ms/step
Epoch 21/50
16/16 - 0s - loss: 59.8558 - val_loss: 81.4217 - 60ms/epoch - 4ms/step
Epoch 22/50
16/16 - 0s - loss: 82.5207 - val_loss: 81.3406 - 61ms/epoch - 4ms/step
Epoch 23/50
16/16 - 0s - loss: 62.0714 - val_loss: 68.8735 - 62ms/epoch - 4ms/step
Epoch 24/50
16/16 - 0s - loss: 67.6048 - val_loss: 68.9049 - 64ms/epoch - 4ms/step
Epoch 25/50
16/16 - 0s - loss: 58.3201 - val_loss: 68.8365 - 63ms/epoch - 4ms/step
Epoch 26/50
16/16 - 0s - loss: 64.1019 - val_loss: 71.2896 - 70ms/epoch - 4ms/step
Epoch 27/50
16/16 - 0s - loss: 62.1012 - val_loss: 67.7988 - 61ms/epoch - 4ms/step
Epoch 28/50
16/16 - 0s - loss: 67.0260 - val_loss: 65.0869 - 64ms/epoch - 4ms/step
Epoch 29/50
16/16 - 0s - loss: 58.4015 - val_loss: 63.9911 - 62ms/epoch - 4ms/step
Epoch 30/50
16/16 - 0s - loss: 60.5792 - val_loss: 61.8820 - 69ms/epoch - 4ms/step
Epoc

Epoch 17/50
16/16 - 0s - loss: 55.6624 - val_loss: 59.7715 - 87ms/epoch - 5ms/step
Epoch 18/50
16/16 - 0s - loss: 55.9263 - val_loss: 72.5703 - 78ms/epoch - 5ms/step
Epoch 19/50
16/16 - 0s - loss: 63.7263 - val_loss: 60.2315 - 77ms/epoch - 5ms/step
Epoch 20/50
16/16 - 0s - loss: 62.8919 - val_loss: 59.2642 - 68ms/epoch - 4ms/step
Epoch 21/50
16/16 - 0s - loss: 55.7237 - val_loss: 59.2417 - 83ms/epoch - 5ms/step
Epoch 22/50
16/16 - 0s - loss: 55.8962 - val_loss: 60.9576 - 78ms/epoch - 5ms/step
Epoch 23/50
16/16 - 0s - loss: 58.9719 - val_loss: 59.2339 - 85ms/epoch - 5ms/step
Epoch 24/50
16/16 - 0s - loss: 58.5862 - val_loss: 62.1279 - 67ms/epoch - 4ms/step
Epoch 25/50
16/16 - 0s - loss: 54.9961 - val_loss: 60.4856 - 82ms/epoch - 5ms/step
Epoch 26/50
16/16 - 0s - loss: 55.9476 - val_loss: 62.8416 - 65ms/epoch - 4ms/step
Epoch 27/50
16/16 - 0s - loss: 54.2177 - val_loss: 59.9513 - 67ms/epoch - 4ms/step
Epoch 28/50
16/16 - 0s - loss: 55.4551 - val_loss: 78.1847 - 68ms/epoch - 4ms/step
Epoc

In [84]:
len(MSE_list)

50

In [83]:
MSE_list

[120.79628358905525,
 115.34411382793884,
 110.99221840475028,
 115.98207684744582,
 110.11194674633734,
 130.80762927425587,
 119.45826536632875,
 121.3842184886661,
 150.73704009808046,
 128.87671728064757,
 110.34668668194551,
 122.91643885803349,
 117.7685522085296,
 113.97201243276093,
 111.10136684624409,
 111.00704345804694,
 113.16345082325056,
 118.3202792355226,
 113.55119082795694,
 110.30049313058912,
 109.56694444182936,
 109.77822437311413,
 111.58706082614701,
 128.42125335347836,
 111.60457010703865,
 112.4980406385525,
 111.19602578247675,
 109.3153909111746,
 112.08298381134935,
 116.70496518272296,
 109.8206592683531,
 112.90554718293585,
 99.39895361368283,
 82.76606517850384,
 71.23090812155334,
 69.13364793659227,
 74.33192398698777,
 64.29019897555769,
 63.82661337348454,
 63.902443647936565,
 62.55899900259018,
 81.56977398082957,
 61.64963712751215,
 62.06433088363119,
 63.660994887705506,
 61.66864524490469,
 60.389072195725404,
 56.98605128344915,
 56.5341720

In [85]:
mean= np.mean(MSE_list)
mean

98.68602776602138

In [88]:
standard_deviation = np.std(MSE_list)
standard_deviation

25.328164847448765

In [ ]:
print("mean squared error")
print("Standard deviation:", standard_deviation)

### Normlize the data

In [15]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [16]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8